## Configurations

In [1]:
#Needed Imports
import psycopg2

%matplotlib inline
import os
import sys
import configparser
import numpy as np
import pandas as pd

import math

import pandas as pd
from geopandas import gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point, MultiPoint, Polygon, MultiPolygon, GeometryCollection, box


In [2]:
#Setting basic configurations

#The config file especially contains the access information for the DB server used to store detection results
path_to_config_file = r"C:\school\Diss\FINT_CH\Config"
ini_config_file = os.path.join(path_to_config_file, "FINTCH_config.ini")

cfg = configparser.ConfigParser()
cfg._interpolation = configparser.ExtendedInterpolation()
cfg.read(ini_config_file)

kantonsliste_path = r"C:\school\Diss\FINT_CH\Data\Original_Data\kantone_info.csv"

epsg = "2056" #LV95/CH1903+
crs = {'init': "epsg:"+epsg} #http://www.spatialreference.org/ref/epsg/2056/


## Data Acquisition

In [3]:
#Creating DB connection
con = psycopg2.connect(host=cfg.get("AP07__db","host"), dbname=cfg.get("AP07__db","dbname"), user=cfg.get("AP07__db","user"), password=cfg.get("AP07__db","password"))

#Reading detected trees with information on plots

sql_trees = """
SELECT
    perimeter_id, flaeche_id,
    SUM(tree_ps1) AS stems_in_plot_ps1,
    SUM(tree_ps2) AS stems_in_plot_ps2,
    SUM(tree_ps3) AS stems_in_plot_ps3,
    SUM(tree_ps4) AS stems_in_plot_ps4,
    SUM(tree_ps5) AS stems_in_plot_ps5,
    SUM(tree_ps6) AS stems_in_plot_ps6,
    SUM(tree_ps7) AS stems_in_plot_ps7,
    SUM(tree_ps8) AS stems_in_plot_ps8,
    SUM(tree_ps9) AS stems_in_plot_ps9,
    SUM(tree_ps10) AS stems_in_plot_ps10,
    SUM(tree_ps11) AS stems_in_plot_ps11,
    SUM(tree_ps12) AS stems_in_plot_ps12,
    SUM(tree_ps13) AS stems_in_plot_ps13,
    SUM(tree_ps14) AS stems_in_plot_ps14,
    SUM(tree_ps15) AS stems_in_plot_ps15,
    SUM(tree_ps16) AS stems_in_plot_ps16,
    SUM(tree_ps17) AS stems_in_plot_ps17,
    SUM(tree_ps18) AS stems_in_plot_ps18,
    SUM(tree_ps19) AS stems_in_plot_ps19,
    SUM(tree_ps1*bas) AS bas_detected_ps1,
    SUM(tree_ps2*bas) AS bas_detected_ps2,
    SUM(tree_ps3*bas) AS bas_detected_ps3,
    SUM(tree_ps4*bas) AS bas_detected_ps4,
    SUM(tree_ps5*bas) AS bas_detected_ps5,
    SUM(tree_ps6*bas) AS bas_detected_ps6,
    SUM(tree_ps7*bas) AS bas_detected_ps7,
    SUM(tree_ps8*bas) AS bas_detected_ps8,
    SUM(tree_ps9*bas) AS bas_detected_ps9,
    SUM(tree_ps10*bas) AS bas_detected_ps10, 
    SUM(tree_ps11*bas) AS bas_detected_ps11, 
    SUM(tree_ps12*bas) AS bas_detected_ps12, 
    SUM(tree_ps13*bas) AS bas_detected_ps13, 
    SUM(tree_ps14*bas) AS bas_detected_ps14, 
    SUM(tree_ps15*bas) AS bas_detected_ps15, 
    SUM(tree_ps16*bas) AS bas_detected_ps16, 
    SUM(tree_ps17*bas) AS bas_detected_ps17, 
    SUM(tree_ps18*bas) AS bas_detected_ps18, 
    SUM(tree_ps19*bas) AS bas_detected_ps19, 
    MIN(prtid) AS prtid, MIN(plot_area) AS plot_area ,
    MIN(source_id) AS source_id,
    MIN(slope) AS slope,
    MIN(slope_sin) AS slope_sin,
    MIN(slope_class) AS slope_class,
    MIN(aspect) AS aspect,
    MIN(aspect_sin) AS aspect_sin,
    MIN(aspect_cos) AS aspect_cos,
    MIN(aspect_class) AS aspect_class,
    MIN(northness) AS northness,
    MIN(eastness) AS eastness,
    MIN(veg_code) AS veg_code,
    MIN(z) AS z,
    MIN(veg_subcode) AS veg_subcode,
    MIN(veg_de) AS veg_de,
    MIN(hdom) AS hdom,
    MIN(dg) AS dg,
    MIN(nh) AS nh,
    MIN(fst) AS fst,
    MIN(hdom50) AS hdom50, MIN(dg50) AS dg50, MIN(nh50) AS nh50, MIN(fst50) AS fst50, MIN(dg_ks) AS dg_ks,MIN(dg_us) AS dg_us,MIN(dg_ms) AS dg_ms,MIN(dg_os) AS dg_os,MIN(dg_ueb) AS dg_ueb,MIN(dg_ks50) AS dg_ks50,MIN(dg_us50) AS dg_us50,MIN(dg_ms50) AS dg_ms50,MIN(dg_os50) AS dg_os50,MIN(dg_ueb50) AS dg_ueb50, 
    radius,
    MIN(geom) AS geom
  FROM
    (SELECT ft.gid, ft.x, ft.y, ft.hoehe, ft.dominanz, ft.bhd, ft.geom,
          ft.parameterset_id, ft.perimeter_id, ft.flaeche_id,
              CASE WHEN ft.parameterset_id = 2 THEN 1 ELSE 0 END AS Tree_ps1,
              CASE WHEN ft.parameterset_id = 2 THEN 1 ELSE 0 END AS Tree_ps2,
              CASE WHEN ft.parameterset_id = 3 THEN 1 ELSE 0 END AS Tree_ps3,
              CASE WHEN ft.parameterset_id = 4 THEN 1 ELSE 0 END AS Tree_ps4,
              CASE WHEN ft.parameterset_id = 5 THEN 1 ELSE 0 END AS Tree_ps5,
              CASE WHEN ft.parameterset_id = 6 THEN 1 ELSE 0 END AS Tree_ps6,
              CASE WHEN ft.parameterset_id = 7 THEN 1 ELSE 0 END AS Tree_ps7,
              CASE WHEN ft.parameterset_id = 8 THEN 1 ELSE 0 END AS Tree_ps8,
              CASE WHEN ft.parameterset_id = 9 THEN 1 ELSE 0 END AS Tree_ps9,
              CASE WHEN ft.parameterset_id = 10 THEN 1 ELSE 0 END AS Tree_ps10,
              CASE WHEN ft.parameterset_id = 11 THEN 1 ELSE 0 END AS Tree_ps11,
              CASE WHEN ft.parameterset_id = 12 THEN 1 ELSE 0 END AS Tree_ps12,
              CASE WHEN ft.parameterset_id = 13 THEN 1 ELSE 0 END AS Tree_ps13,
              CASE WHEN ft.parameterset_id = 14 THEN 1 ELSE 0 END AS Tree_ps14,
              CASE WHEN ft.parameterset_id = 15 THEN 1 ELSE 0 END AS Tree_ps15,
              CASE WHEN ft.parameterset_id = 16 THEN 1 ELSE 0 END AS Tree_ps16,
              CASE WHEN ft.parameterset_id = 17 THEN 1 ELSE 0 END AS Tree_ps17,
              CASE WHEN ft.parameterset_id = 18 THEN 1 ELSE 0 END AS Tree_ps18,
              CASE WHEN ft.parameterset_id = 19 THEN 1 ELSE 0 END AS Tree_ps19,
              (ft.bhd/200)^2*pi() AS bas,
          p.gid AS prtid, (12.62)^2*pi() AS plot_area, p.source_id,
          p.slope, p.slope_sin, p.slope_class, p.aspect, p.aspect_sin, p.aspect_cos, p.aspect_class, p.northness, p.eastness, p.z, p.veg_code, p.veg_subcode, p.veg_de, p.hdom, p.dg, p.nh, p.fst, radius,
          p.hdom50, p.dg50, p.nh50, p.fst50, p.dg_ks,p.dg_us,p.dg_ms,p.dg_os,p.dg_ueb,p.dg_ks50,p.dg_us50,p.dg_ms50,p.dg_os50,p.dg_ueb50
        FROM fintch.diss_tree_detected AS ft
        LEFT JOIN fintch.diss_perimeter AS p ON ft.perimeter_id = p.perimeter_id AND ft.flaeche_id = p.flaeche_id
         CROSS JOIN (SELECT *
         --FROM json_to_recordset('[{"radius":-5},{"radius":-4},{"radius":-3},{"radius":-2},{"radius":-1},{"radius":0},{"radius":1},{"radius":2},{"radius":3},{"radius":4},{"radius":5},{"radius":6},{"radius":7}]')
         --FROM json_to_recordset('[{"radius":-2},{"radius":-1},{"radius":0},{"radius":1},{"radius":2}]')
         FROM json_to_recordset('[{"radius":0}]')

        AS x("radius" int)) AS rad
        WHERE p.source_id IN (1)--2:GR1  ,3,4,5
        --AND ft.parameterset_id IN ()
        AND ST_Intersects(ft.geom,ST_Buffer(ST_Centroid(p.geom),(12.62+radius)))
     ) AS t

    GROUP BY  perimeter_id, flaeche_id, radius
            ;
"""

fint_trees_db = gpd.GeoDataFrame.from_postgis(sql_trees, con, geom_col='geom' )
len(fint_trees_db)

5915

In [4]:
#Creating DB connection
con = psycopg2.connect(host=cfg.get("AP07__db","host"), dbname=cfg.get("AP07__db","dbname"), user=cfg.get("AP07__db","user"), password=cfg.get("AP07__db","password"))

sql_method_combinations = """SELECT id, size, combination, rmse, dr, dr_delta
FROM fintch.diss_method_combinations;"""
    
    
method_combinations_db = pd.read_sql_query(sql_method_combinations, con)
len(method_combinations_db)

csv_out_path = r"C:\school\Diss\FINT_CH\Data\method_combinations_lfi.csv"
method_combinations_db.to_csv(csv_out_path, sep=";", header=True, quotechar='"', index=False)


In [5]:
#Reading plot information only
sql_envelopes = """
SELECT gid, geom, slope, slope_sin, slope_class, aspect, aspect_sin, aspect_cos, aspect_class, northness, eastness, z,  veg_code, veg_subcode, veg_de, hdom, dg, nh, fst, perimeter_id, source_id, flaeche_id
    FROM fintch.diss_perimeter;
"""
    
fint_envelopes_db = gpd.GeoDataFrame.from_postgis(sql_envelopes, con, geom_col='geom' )
len(fint_envelopes_db)    

20659

In [6]:
#Read list with infos about cantons
kantonsliste_df = pd.read_csv(kantonsliste_path, delimiter=";")

In [7]:
#Labels for the detection method entries
parameterset_lookup_label_dict = {
 1: '1m',
 2: '1.5m',
 3: '2m',
 4: '1m, sigma 1, size 3',
 5: '1m, sigma 1, size 5',
 6: '1m, sigma 1, size 7',
 7: '1m, sigma 2, size 3',
 8: '1m, sigma 2, size 5',
 9: '1m, sigma 2, size 7',
 10: '1m, sigma 3, size 3',
 11: '1m, sigma 3, size 5',
 12: '1m, sigma 3, size 7',
 13: '1.5m, sigma 2, size 3',
 14: '1.5m, sigma 2, size 5',
 15: '1.5m, sigma 2, size 7',
 16: 'combined 1m/1.5m/1m, sig2, r3',
 17: 'combined 1m/2m/1m, sig2, r5',
 18: 'combined 1m/1.5m/1m, sig1, r3/1m, sig2, r3',
 19: 'combined 1m/1.5m/1m, sig1, r3/1m, sig2, r3/1m, sig3, r3',
} 



In [8]:
#Labels for the reference data sources
source_label_dict = {"source_id": [1,2,3,4,5], "source_label": ["LFI4", "GR1 1996-2009", "GR2 2010-2019 ", "FL", "BL"]}
source_label_df = pd.DataFrame(source_label_dict)

In [9]:
#reading aggregated reference data for the plots
plot_ref_csv_path = r".\Data\reference_plots.csv"
plot_ref_csv_path = r".\Data\reference_plots_lfi_only.csv"
plot_ref_df =  pd.read_csv(plot_ref_csv_path, delimiter=";")


## Data Preparation

In [10]:
fint_trees_db.columns

Index(['perimeter_id', 'flaeche_id', 'stems_in_plot_ps1', 'stems_in_plot_ps2',
       'stems_in_plot_ps3', 'stems_in_plot_ps4', 'stems_in_plot_ps5',
       'stems_in_plot_ps6', 'stems_in_plot_ps7', 'stems_in_plot_ps8',
       'stems_in_plot_ps9', 'stems_in_plot_ps10', 'stems_in_plot_ps11',
       'stems_in_plot_ps12', 'stems_in_plot_ps13', 'stems_in_plot_ps14',
       'stems_in_plot_ps15', 'stems_in_plot_ps16', 'stems_in_plot_ps17',
       'stems_in_plot_ps18', 'stems_in_plot_ps19', 'bas_detected_ps1',
       'bas_detected_ps2', 'bas_detected_ps3', 'bas_detected_ps4',
       'bas_detected_ps5', 'bas_detected_ps6', 'bas_detected_ps7',
       'bas_detected_ps8', 'bas_detected_ps9', 'bas_detected_ps10',
       'bas_detected_ps11', 'bas_detected_ps12', 'bas_detected_ps13',
       'bas_detected_ps14', 'bas_detected_ps15', 'bas_detected_ps16',
       'bas_detected_ps17', 'bas_detected_ps18', 'bas_detected_ps19', 'prtid',
       'plot_area', 'source_id', 'slope', 'slope_sin', 'slope_class', '

In [11]:
#Clean anomalous terrain values for aspect in flat places (aspect==-9999)
fint_trees_db["aspect_cos"] = fint_trees_db.apply(lambda x: 1 if x["aspect"]==-9999 else x["aspect_cos"] ,axis=1)
fint_trees_db["aspect_sin"] = fint_trees_db.apply(lambda x: 0 if x["aspect"]==-9999 else x["aspect_sin"] ,axis=1)
fint_trees_db["northness"] = fint_trees_db.apply(lambda x: x["aspect_cos"]*x["slope_sin"] if x["aspect"]==-9999 else x["northness"] ,axis=1)
fint_trees_db["eastness"] = fint_trees_db.apply(lambda x: x["aspect_sin"]*x["slope_sin"] if x["aspect"]==-9999 else x["eastness"] ,axis=1)
fint_trees_db["aspect"] = fint_trees_db.apply(lambda x: 0 if x["aspect"]==-9999 else x["aspect"] ,axis=1)


In [12]:
fint_trees = fint_trees_db

In [13]:
#Add "source_label" to FINT-CH trees
fint_trees = fint_trees.merge(source_label_df, left_on=["source_id"], right_on=["source_id"],how="left")

In [14]:
#Calculate values per ha and set placeholders for empty detections
#99999999 is used, if atual method of detection and column don't match --> need to aggregate with MIN and to correctly treat value later
for i in range(1,20,1):
    idx = str(i)
    fint_trees["stems_per_ha_ps"+idx] = fint_trees.apply(lambda x: (x["stems_in_plot_ps"+idx]/(x["plot_area"]/10000)) , axis=1)
    fint_trees["bas_per_ha_ps"+idx] = fint_trees.apply(lambda x: (x["bas_detected_ps"+idx]/(x["plot_area"]/10000)) , axis=1)
    

In [15]:
plot_ref_df.columns

Index(['OBJECTID', 'plot_id', 'x_lv95', 'y_lv95', 'waldrand',
       'bestandesgrenze', 'nbr_trees', 'nbr_trees_os', 'bas', 'year',
       'density_actual', 'density_actual_all', 'bas_per_ha',
       'density_actual_LFI', 'nbr_trees_LFI', 'source_id', 'geometry',
       'index_right', 'KANTONSNUM', 'VHM_Year_Min', 'VHM_Year_Max',
       'file_name_als', 'VHM_Tile_Year'],
      dtype='object')

In [16]:
fint_trees.columns

Index(['perimeter_id', 'flaeche_id', 'stems_in_plot_ps1', 'stems_in_plot_ps2',
       'stems_in_plot_ps3', 'stems_in_plot_ps4', 'stems_in_plot_ps5',
       'stems_in_plot_ps6', 'stems_in_plot_ps7', 'stems_in_plot_ps8',
       ...
       'stems_per_ha_ps15', 'bas_per_ha_ps15', 'stems_per_ha_ps16',
       'bas_per_ha_ps16', 'stems_per_ha_ps17', 'bas_per_ha_ps17',
       'stems_per_ha_ps18', 'bas_per_ha_ps18', 'stems_per_ha_ps19',
       'bas_per_ha_ps19'],
      dtype='object', length=115)

In [17]:
#Derive codes for expert determined forest structure type from original project
fint_trees["fst_nh"] = fint_trees["fst"]//100
fint_trees["fst_dg"] = (fint_trees["fst"]//10)%10
fint_trees["fst_hdom"] = fint_trees["fst"]%10

In [18]:
def set_ps_list(x,source_column="stems_per_ha",index_column="parameterset_id"):
    vect = [99999999]*19
    vect[x[index_column]-1]=x[source_column]
    return vect

In [19]:
#Aggregate values per plot and parameterset
fint_trees_grouped = fint_trees #Grouping is now done in Query

In [20]:
#Add plot reference data
#fint_trees_grouped = fint_trees_grouped.merge(plot_ref_df, left_on=["perimeter_id"], right_on=["OBJECTID"],how="left",suffixes=["","_ref"])
fint_trees_grouped = fint_trees_grouped.merge(plot_ref_df, left_on=["flaeche_id","source_id"], right_on=["plot_id","source_id"],how="left",suffixes=["","_ref"])

In [21]:
fint_trees_grouped = fint_trees_grouped[np.invert(np.isnan(fint_trees_grouped["density_actual"]))]

In [22]:
fint_trees_grouped.columns

Index(['perimeter_id', 'flaeche_id', 'stems_in_plot_ps1', 'stems_in_plot_ps2',
       'stems_in_plot_ps3', 'stems_in_plot_ps4', 'stems_in_plot_ps5',
       'stems_in_plot_ps6', 'stems_in_plot_ps7', 'stems_in_plot_ps8',
       ...
       'bas_per_ha', 'density_actual_LFI', 'nbr_trees_LFI', 'geometry',
       'index_right', 'KANTONSNUM', 'VHM_Year_Min', 'VHM_Year_Max',
       'file_name_als', 'VHM_Tile_Year'],
      dtype='object', length=140)

In [23]:
fint_trees_grouped.rename(columns={"bas":"bas_ref", "bas_per_ha":"bas_per_ha_ref"}, inplace=True)

In [24]:
#Removing columns that are unneeded or not aggregated properly (e.g. individual tree specific)
# fint_trees_grouped.drop(columns=["IsMittelschicht","IsOberschicht","IsUnterschicht","bas","bas_per_ha","bas_sum","bhd","dominanz","gid",
#                  "hoehe","prtid","stems_in_plot","stems_per_ha","x","y","OBJECTID","geometry",
#                 "stems_in_plot_ms","stems_in_plot_os","stems_in_plot_us","stems_ms_per_ha","stems_os_per_ha","stems_us_per_ha",
#                 "parameterset_id","parameterset_label","plot_id",
#                                 ],inplace=True)


fint_trees_grouped.drop(columns=["OBJECTID",
"plot_id",
"prtid",
"geom",
"geometry",
                                ],inplace=True)




Determining the detection variant with the smallest squered difference between detected and actual value. If there are two methods with the same minimal value, the method with simpler processing (resize over gauss, gauss over combination) are prioritised. For the gauss filtering, the sigma 2 variants are prioritized

In [25]:
#Calculate squared difference based on actual density with all trees
for i in range(1,20,1):
    idx = str(i)
    fint_trees_grouped["stems_per_ha_rs_all_ps"+idx] = fint_trees_grouped.apply(lambda x: (x["stems_per_ha_ps"+idx]-x["density_actual_all"] if x["stems_per_ha_ps"+idx]!=99999999 else x["density_actual_all"] )**2 , axis=1)



In [26]:
def determine_min_class_flex(x, input_prefix="stems_per_ha_rs_", input_classes=[1,2,3,7,8,9,4,5,6,10,11,12,13,14,15,16,17,18,19]):
    input_columns = [input_prefix+"ps"+str(i) for i in input_classes]
    min_val = x[input_columns].min()
    for i in input_classes:
        if x[input_prefix+"ps"+str(i)] == min_val:
            return pd.Series([int(i),min_val],index=["parameterset_min","rmse_min"])
    
#Determine method with minimal squared difference
#lfi_trees_grouped["parameterset_min_reduced"] = lfi_trees_grouped.apply(determine_min_class_redux, input_prefix="stems_per_ha_rs_", axis=1)    


In [27]:
plot_ref_df.columns

Index(['OBJECTID', 'plot_id', 'x_lv95', 'y_lv95', 'waldrand',
       'bestandesgrenze', 'nbr_trees', 'nbr_trees_os', 'bas', 'year',
       'density_actual', 'density_actual_all', 'bas_per_ha',
       'density_actual_LFI', 'nbr_trees_LFI', 'source_id', 'geometry',
       'index_right', 'KANTONSNUM', 'VHM_Year_Min', 'VHM_Year_Max',
       'file_name_als', 'VHM_Tile_Year'],
      dtype='object')

In [28]:
#Determine method with minimal squared difference
fint_trees_grouped[["parameterset_min_all","rmse_min_all"]] = fint_trees_grouped.apply(determine_min_class_flex, input_prefix="stems_per_ha_rs_all_", axis=1)
fint_trees_grouped["parameterset_min_all"] = fint_trees_grouped["parameterset_min_all"].astype("int32")

In [29]:
#Copy value from minimal method to separate column
min_col = fint_trees_grouped["parameterset_min_all"].apply(lambda x: "stems_per_ha_rs_all_ps"+str(x)) 
fint_trees_grouped["stems_per_ha_rs_min_all"] =  [fint_trees_grouped.loc[idx, min_col.iloc[i]] for i, idx in enumerate(min_col.index.values)]

In [30]:
#Calculate squared difference based on actual density with trees of the upper layer
for i in range(1,20,1):
    idx = str(i)
    fint_trees_grouped["stems_per_ha_rs_ps"+idx] = fint_trees_grouped.apply(lambda x: (x["stems_per_ha_ps"+idx]-x["density_actual"] if x["stems_per_ha_ps"+idx]!=99999999 else x["density_actual"] )**2 , axis=1)


In [31]:
#Determine method with minimal squared difference
fint_trees_grouped[["parameterset_min","rmse_min"]] = fint_trees_grouped.apply(determine_min_class_flex, input_prefix="stems_per_ha_rs_", axis=1)
fint_trees_grouped["parameterset_min"] = fint_trees_grouped["parameterset_min"].astype("int32")

In [32]:
#Copy value from minimal method to separate column
min_col = fint_trees_grouped["parameterset_min"] .apply(lambda x: "stems_per_ha_rs_ps"+str(x)) 
fint_trees_grouped["stems_per_ha_rs_min"] =  [fint_trees_grouped.loc[idx, min_col.iloc[i]] for i, idx in enumerate(min_col.index.values)]


In [33]:
#Add label for min method
fint_trees_grouped["parameterset_min_label"] = fint_trees_grouped["parameterset_min"].apply(lambda x: parameterset_lookup_label_dict[int(x)] )
fint_trees_grouped["parameterset_min_all_label"] = fint_trees_grouped["parameterset_min_all"].apply(lambda x: parameterset_lookup_label_dict[int(x)] )

In [34]:
csv_out_path = r"C:\school\Diss\FINT_CH\Data\detection_aggregated_all_radius.csv"
fint_trees_grouped.to_csv(csv_out_path, sep=";", header=True, quotechar='"', index=False)


In [35]:
#Determine circle size with lowest rmse
fint_trees_grouped["rmse_min_global"] = fint_trees_grouped.groupby(["perimeter_id", "flaeche_id"])["rmse_min"].transform('min')
fint_trees_grouped["is_min_radius"] = fint_trees_grouped.apply(lambda x: x["radius"] if x["rmse_min_global"]==x["rmse_min"] else -9999, axis=1)
fint_trees_grouped["min_radius"] = fint_trees_grouped.groupby(["perimeter_id", "flaeche_id"])["is_min_radius"].transform('max')
print(len(fint_trees_grouped))
#Filter to retain only one record per plot
fint_trees_grouped_level2 = fint_trees_grouped[fint_trees_grouped["radius"]==fint_trees_grouped["min_radius"]].reset_index(drop=True)
print(len(fint_trees_grouped_level2))
print(len(fint_trees_grouped_level2["perimeter_id"].unique()))


5913
5913
5913


In [36]:
actual = "density_actual"
actual_all = "density_actual_all"
predicted = "stems_per_ha_ps"

for i in range(1,20,1):
    idx = str(i)
    fint_trees_grouped["dr_ps"+idx] = fint_trees_grouped.apply(lambda g: (g[predicted+idx]/g[actual] if g[actual]>0 else (1 if g[predicted+idx] == g[actual] else 0)) if g[predicted+idx]!=99999999 else 0 , axis=1)
    fint_trees_grouped["dr_all_ps"+idx] = fint_trees_grouped.apply(lambda g: (g[predicted+idx]/g[actual_all] if g[actual_all]>0 else (1 if g[predicted+idx] == g[actual_all] else 0)) if g[predicted+idx]!=99999999 else 0 , axis=1)


In [37]:
#Add detection rates per method
actual = "density_actual"
actual_all = "density_actual_all"
predicted = "stems_per_ha_ps"

for i in range(1,20,1):
    idx = str(i)
    fint_trees_grouped_level2["dr_ps"+idx] = fint_trees_grouped_level2.apply(lambda g: (g[predicted+idx]/g[actual] if g[actual]>0 else (1 if g[predicted+idx] == g[actual] else 0)) if g[predicted+idx]!=99999999 else 0 , axis=1)
    fint_trees_grouped_level2["dr_all_ps"+idx] = fint_trees_grouped_level2.apply(lambda g: (g[predicted+idx]/g[actual_all] if g[actual_all]>0 else (1 if g[predicted+idx] == g[actual_all] else 0)) if g[predicted+idx]!=99999999 else 0 , axis=1)


In [38]:
#Reorder columns 
fint_trees_grouped_level2 = fint_trees_grouped_level2[[
    "perimeter_id","flaeche_id","source_id","source_label","radius",

"slope","slope_class","slope_sin","aspect","aspect_class","aspect_cos","aspect_sin","northness","eastness","z","veg_code","veg_de","veg_subcode","fst","fst_dg","fst_hdom","fst_nh","dg","hdom","nh", "hdom50", "dg50", "nh50", "fst50", "dg_ks", "dg_us", "dg_ms", "dg_os", "dg_ueb", "dg_ks50", "dg_us50", "dg_ms50", "dg_os50", "dg_ueb50",

"x_lv95","y_lv95","waldrand","bestandesgrenze","nbr_trees","bas_ref","year","density_actual","density_actual_all","bas_per_ha_ref","KANTONSNUM","VHM_Year_Min","VHM_Year_Max","VHM_Tile_Year","plot_area",
"stems_per_ha_ps1","stems_per_ha_ps2","stems_per_ha_ps3","stems_per_ha_ps4","stems_per_ha_ps5","stems_per_ha_ps6","stems_per_ha_ps7","stems_per_ha_ps8","stems_per_ha_ps9","stems_per_ha_ps10","stems_per_ha_ps11","stems_per_ha_ps12","stems_per_ha_ps13","stems_per_ha_ps14","stems_per_ha_ps15","stems_per_ha_ps16","stems_per_ha_ps17","stems_per_ha_ps18","stems_per_ha_ps19",
"bas_per_ha_ps1","bas_per_ha_ps2","bas_per_ha_ps3","bas_per_ha_ps4","bas_per_ha_ps5","bas_per_ha_ps6","bas_per_ha_ps7","bas_per_ha_ps8","bas_per_ha_ps9","bas_per_ha_ps10","bas_per_ha_ps11","bas_per_ha_ps12","bas_per_ha_ps13","bas_per_ha_ps14","bas_per_ha_ps15","bas_per_ha_ps16","bas_per_ha_ps17","bas_per_ha_ps18","bas_per_ha_ps19",
"dr_ps1","dr_ps2","dr_ps3","dr_ps4","dr_ps5","dr_ps6","dr_ps7","dr_ps8","dr_ps9","dr_ps10","dr_ps11","dr_ps12","dr_ps13","dr_ps14","dr_ps15","dr_ps16","dr_ps17","dr_ps18","dr_ps19",
"dr_all_ps1","dr_all_ps2","dr_all_ps3","dr_all_ps4","dr_all_ps5","dr_all_ps6","dr_all_ps7","dr_all_ps8","dr_all_ps9","dr_all_ps10","dr_all_ps11","dr_all_ps12","dr_all_ps13","dr_all_ps14","dr_all_ps15","dr_all_ps16","dr_all_ps17","dr_all_ps18","dr_all_ps19",   
"bas_detected_ps1","bas_detected_ps2","bas_detected_ps3","bas_detected_ps4","bas_detected_ps5","bas_detected_ps6","bas_detected_ps7","bas_detected_ps8","bas_detected_ps9","bas_detected_ps10","bas_detected_ps11","bas_detected_ps12","bas_detected_ps13","bas_detected_ps14","bas_detected_ps15","bas_detected_ps16","bas_detected_ps17","bas_detected_ps18","bas_detected_ps19",
"stems_in_plot_ps1","stems_in_plot_ps2","stems_in_plot_ps3","stems_in_plot_ps4","stems_in_plot_ps5","stems_in_plot_ps6","stems_in_plot_ps7","stems_in_plot_ps8","stems_in_plot_ps9","stems_in_plot_ps10","stems_in_plot_ps11","stems_in_plot_ps12","stems_in_plot_ps13","stems_in_plot_ps14","stems_in_plot_ps15","stems_in_plot_ps16","stems_in_plot_ps17","stems_in_plot_ps18","stems_in_plot_ps19",
"stems_per_ha_rs_all_ps1","stems_per_ha_rs_all_ps2","stems_per_ha_rs_all_ps3","stems_per_ha_rs_all_ps4","stems_per_ha_rs_all_ps5","stems_per_ha_rs_all_ps6","stems_per_ha_rs_all_ps7","stems_per_ha_rs_all_ps8","stems_per_ha_rs_all_ps9","stems_per_ha_rs_all_ps10","stems_per_ha_rs_all_ps11","stems_per_ha_rs_all_ps12","stems_per_ha_rs_all_ps13","stems_per_ha_rs_all_ps14","stems_per_ha_rs_all_ps15","stems_per_ha_rs_all_ps16","stems_per_ha_rs_all_ps17","stems_per_ha_rs_all_ps18","stems_per_ha_rs_all_ps19",
"stems_per_ha_rs_ps1","stems_per_ha_rs_ps2","stems_per_ha_rs_ps3","stems_per_ha_rs_ps4","stems_per_ha_rs_ps5","stems_per_ha_rs_ps6","stems_per_ha_rs_ps7","stems_per_ha_rs_ps8","stems_per_ha_rs_ps9","stems_per_ha_rs_ps10","stems_per_ha_rs_ps11","stems_per_ha_rs_ps12","stems_per_ha_rs_ps13","stems_per_ha_rs_ps14","stems_per_ha_rs_ps15","stems_per_ha_rs_ps16","stems_per_ha_rs_ps17","stems_per_ha_rs_ps18","stems_per_ha_rs_ps19",
"parameterset_min","parameterset_min_label",
"parameterset_min_all","parameterset_min_all_label",
]].reset_index(drop=True)



In [39]:
csv_out_path = r"C:\school\Diss\FINT_CH\Data\detection_aggregated.csv"
fint_trees_grouped_level2.to_csv(csv_out_path, sep=";", header=True, quotechar='"', index=False)


In [40]:
#Only select data from LFI
lfi_trees_grouped = fint_trees_grouped_level2[fint_trees_grouped_level2["source_id"]==1].reset_index(drop=True)

csv_out_path = r"C:\school\Diss\FINT_CH\Data\detection_aggregated_lfi.csv"
lfi_trees_grouped.to_csv(csv_out_path, sep=";", header=True, quotechar='"', index=False)


In [41]:
#Output base structure for markdown documentation of columns (see Data_Analysis.ipynb)
col_string = ""
for c in lfi_trees_grouped.columns:
    print("| ",c," | ", lfi_trees_grouped.dtypes[c]," | | | | |")
    col_string+='"'+c+'", '
print(col_string)

|  perimeter_id  |  int64  | | | | |
|  flaeche_id  |  int64  | | | | |
|  source_id  |  int64  | | | | |
|  source_label  |  object  | | | | |
|  radius  |  int64  | | | | |
|  slope  |  float64  | | | | |
|  slope_class  |  int64  | | | | |
|  slope_sin  |  float64  | | | | |
|  aspect  |  float64  | | | | |
|  aspect_class  |  int64  | | | | |
|  aspect_cos  |  float64  | | | | |
|  aspect_sin  |  float64  | | | | |
|  northness  |  float64  | | | | |
|  eastness  |  float64  | | | | |
|  z  |  float64  | | | | |
|  veg_code  |  int64  | | | | |
|  veg_de  |  object  | | | | |
|  veg_subcode  |  int64  | | | | |
|  fst  |  int64  | | | | |
|  fst_dg  |  int64  | | | | |
|  fst_hdom  |  int64  | | | | |
|  fst_nh  |  int64  | | | | |
|  dg  |  int64  | | | | |
|  hdom  |  int64  | | | | |
|  nh  |  int64  | | | | |
|  hdom50  |  int64  | | | | |
|  dg50  |  int64  | | | | |
|  nh50  |  int64  | | | | |
|  fst50  |  int64  | | | | |
|  dg_ks  |  int64  | | | | |
|  dg_us  |  int64  | 